In [388]:
import pandas as pd
from PIL import Image,ImageDraw,ImageFont
import os
import numpy as np
import math

In [389]:
ii = pd.read_csv("193398_individual_importances.csv")

In [390]:
ii.set_index("entity_id",inplace=True)

In [391]:
feature_table = pd.read_csv("193398_feattable.csv")
feature_table = feature_table[feature_table.feature_importance>0.00353]
feature_table.reset_index(drop=True,inplace=True)
top_feats = list(feature_table.feature)

In [392]:
singleton = 3537

In [393]:
singlevars = [
    ii.risk_1.loc[singleton],
    ii.risk_2.loc[singleton],
    ii.risk_3.loc[singleton],
    ii.risk_4.loc[singleton],
    ii.risk_5.loc[singleton]
    ]

In [394]:
singlevars

['ts_id_p1y_trafficstopsbysearchreason_consent_sum',
 'dispatch_id_p1y_dispatchtype_assault_sum',
 'dispatch_id_p1y_dispatchtype_disorder_sum',
 'dispatch_id_p1m_dispatchtype_disorder_sum',
 'dispatch_id_p1y_dispatchinitiatiationtype_ci_sum']

In [395]:
#hotvar = singlevars[0]

#### test matrix

In [396]:
test_matrix = pd.read_csv("193398_test_2014-04-01.csv")
test_matrix.set_index("officer_id",inplace=True)

In [397]:
test_matrix.sort_values(by='score',ascending=False,inplace=True)
labels = test_matrix.outcome
scores = test_matrix.score

In [398]:
test_matrix = test_matrix[test_matrix['ocag_id_all_officerage_max']!=0]

In [399]:
test_matrix['hotvar_imp'] = False

In [400]:
ii_idxs = [item for item in ii.index if item in test_matrix.index]

In [401]:
for i in ii_idxs:    
    featlist = [
        ii.risk_1.loc[i],
        ii.risk_2.loc[i],
        ii.risk_3.loc[i],
        ii.risk_4.loc[i],
        ii.risk_5.loc[i]
    ]
    
    if hotvar in featlist:
        test_matrix.hotvar_imp.loc[i] = True

In [402]:
print min(scores),max(scores)

0.0624336247543 0.145132730406


In [403]:
test_matrix.outcome.value_counts()

0    1309
1     156
Name: outcome, dtype: int64

In [404]:
tmp = test_matrix[['ocag_id_all_officerage_max','outcome']]

In [405]:
prior = float(156)/1309

In [406]:
max(tmp.ocag_id_all_officerage_max) - min(tmp.ocag_id_all_officerage_max)

45.0

In [407]:
test_matrix.hotvar_imp.value_counts()

False    1130
True      335
Name: hotvar_imp, dtype: int64

In [408]:
hotidxs = test_matrix.index[test_matrix.hotvar_imp==True]

## Binning

In [463]:
def binticks(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    tmprange = tmpmax - tmpmin
    step = tmprange / 100
    
    return np.arange(tmpmin,tmpmax+step,step)

### Dot

In [464]:
def adddot(canvas,binned,singleton):
    width = canvas.width
    height = canvas.height
    margin = rectheight / 2
    newheight = height + margin
    
    unicanvas = Image.new('RGBA',(width,newheight),(255,255,255,0))
    unicanvas.paste(canvas,(0,0))
    draw = ImageDraw.Draw(unicanvas,'RGBA')
    
    sbin = binned[singleton]
    #draw.ellipse((sbin*rectwidth,height-margin,sbin*rectwidth+rectwidth,height+margin),outline="yellow",fill=None)
    
    return unicanvas,sbin

# banger plot

In [465]:
def banger(X,cfeat,rectwidth,rectheight,pad):
    bins = binticks(test_matrix,cfeat)
    nbins = len(bins)
    binned = pd.cut(X.loc[:,cfeat],bins=bins,labels=False,include_lowest=True)
    
    px_w = ( nbins - 1 ) * rectwidth # a hack; won't work with all subsets
    binmax = binned.value_counts().max()
    px_h = binmax * rectheight
    
    canvas = Image.new('RGBA',(px_w,px_h),(255,255,255,0))
    draw = ImageDraw.Draw(canvas)
        
    for binn in range(nbins):
        binheight = len(binned[binned==binn])
        xpos = binn * rectwidth
        bbox = [(xpos,px_h-binheight*rectwidth),(xpos+rectwidth,px_h)]
        draw.rectangle(bbox,fill="hsl(0,0%,50%)",outline=None)
        
    wdot,sbin = adddot(canvas,binned,singleton)
    print nbins,sbin
    return wdot,px_w,px_h

In [466]:
pad = 0

In [467]:
rectwidth = 16
rectheight = 16

In [478]:
cfeat = singlevars[4]

In [479]:
im,px_w,px_h = banger(test_matrix,cfeat,rectwidth,rectheight,pad)

101 83


In [480]:
maxheight = 1224
cropbox = [0,px_h-maxheight,px_w,px_h]
im = im.crop(cropbox)
im.save("/Users/damoncrockett/Desktop/"+cfeat+"_bangerNO.png")